In [3]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import WebDriverException
import datetime
import time
import csv

PATH = '/Users/stefan/Documents/Coding/2021 Internship/California Oil Info/Chromedrive/chromedriver'
driver = webdriver.Chrome(PATH)
labels = ['API Number', 'Operator', 'Active Permit', 'Confidentiality Expiration Date', 'Mineral Owner',
                'Well Spud Date','Status Date', 'Well Classifications', 'Well Designation', 'Well Type', 'Bond Number',
                'Lease', 'Interested Federal Agencies', 'Exploratory Well', 'Initial Date of Production', 'Well Number',
                'Well Status','Confidential Well', 'Surface Owner', 'Well Name', 'Dry Hole', 'Directionally Drilled',
                'Field','Section', 'Latitude','County', 'Datum', 'Elevation above Sea Level', 'Location Description',
                'Area(s)', 'Township','Longitude', 'District','Depth measurements referenced to', 'Corner Call',
                'Pool(s)', 'Range', 'B&M', 'Onshore/Offshore','Feet above ground',"Section", "Latitude", "Field",
                "Township",'Longitude', 'County', "Range", "B&M","Corner Call",'Type:', 'Status:', 'Wellbore Spud Date:',
                'Rig Release Date:', 'Completion Date:','Plugback Depth (MD)',
                'Plugback Depth (TVD)',
                'Base of Fresh Water','Top MD:', 'Bottom MD:','Outside Diameter:', 'Inside Diameter:', 'Weight:','Grade/Type:',
                'New Pipe:', 'Install Date', 'Remove Date','Pulled','Burst Pressure','Connection Type', 'Description',
                'Type:','Interval Status:', 'Field:','Area:','Pool Code:', 'Top MD:','Bottom MD:',
                'Formation:', 'Max. Daily Injection Rate:','Max. Cumulative Volume:', 'Type:','Interval Status:', 'Field:',
                'Area:','Pool Code:', 'Top MD:','Bottom MD:','Formation:', 'Max. Daily Injection Rate:','Max. Cumulative Volume:']

excel_labels = ['API Number', 'Operator', 'Active Permit', 'Confidentiality Expiration Date', 'Mineral Owner',
                'Well Spud Date','Status Date', 'Well Classifications', 'Well Designation', 'Well Type', 'Bond Number',
                'Lease', 'Interested Federal Agencies', 'Exploratory Well', 'Initial Date of Production', 'Well Number',
                'Well Status','Confidential Well', 'Surface Owner', 'Well Name', 'Dry Hole', 'Directionally Drilled',
                'Field','Section', 'Latitude','County', 'Datum', 'Elevation above Sea Level', 'Location Description',
                'Area(s)', 'Township','Longitude', 'District','Depth measurements referenced to', 'Corner Call',
                'Pool(s)', 'Range', 'B&M', 'Onshore/Offshore','Feet above ground',"Section (Bottomhole)",
                "Latitude (Bottomhole)", "Field (Bottomhole)", "Township (Bottomhole)",'Longitude (Bottomhole)',
                'County (Bottomhole)', "Range (Bottomhole)", "B&M (Bottomhole)","Corner Call (Bottomhole)",'Type',
                'Status', 'Wellbore Spud Date', 'Rig Release Date', 'Completion Date',
                'Plugback Depth (MD)', 'Plugback Depth (TVD)','Base of Fresh Water','Top MD (Casing Liner)',
                'Bottom MD (Casing Liner)','Outside Diameter (Casing Liner)', 'Inside Diameter (Casing Liner)',
                'Weight (Casing Liner)','Grade/Type (Casing Liner)','New Pipe (Casing Liner)',
                'Install Date (Casing Liner)', 'Remove Date (Casing Liner)','Pulled (Casing Liner)',
                'Burst Pressure (Casing Liner)','Connection Type (Casing Liner)', 'Description (Casing Liner)',
                'Type (Completion Interval 1)','Interval Status (Completion Interval 1)', 'Field (Completion Interval 1)',
                'Area (Completion Interval 1)','Pool Code (Completion Interval 1)', 'Top MD (Completion Interval 1)',
                'Bottom MD (Completion Interval 1)','Formation (Completion Interval 1)',
                'Max Daily Injection Rate (Completion Interval 1)','Max Cumulative Volume (Completion Interval 1)',
                'Type (Completion Interval 2)','Interval Status (Completion Interval 2)', 'Field (Completion Interval 2)',
                'Area (Completion Interval 2)','Pool Code (Completion Interval 2)', 'Top MD (Completion Interval 2)',
                'Bottom MD (Completion Interval 2)','Formation (Completion Interval 2)',
                'Max Daily Injection Rate (Completion Interval 2)','Max Cumulative Volume (Completion Interval 2)',
                'Wellhead Info','Extra Casing Info','Bottom Hole (MD):','Bottom Hole (TVD):','State','Timestamp']

CA_counties = list(range(13, 116, 2)) + list(range(116, 131, 2)) + list((30, 201, 211, 213, 215, 223, 237, 241, 245, 253, 255, 259, 273, 275, 279, 281, 283, 287, 295, 297, 305)) + list(range(216, 231, 2)) + list(range(307, 314))

#Opening first window, will be closed later
driver.get('https://wellstar-public.conservation.ca.gov/Well/Well/Detail?api=0403100002')
try:
    driver.find_element_by_id("AcceptedAgreement").click()
    driver.find_element_by_id("AcceptButton").click()
except:
    pass

def collectandWrite(county):
    incorrect = 0
    for unique in range(1, 99999):
        if (incorrect < 3):
            values = []
            ordered_values =[]
            print('04' + str(county).zfill(3) + str(unique).zfill(5))
            driver.execute_script('''window.open();''')
            driver.switch_to.window(driver.window_handles[1])
            driver.get('https://wellstar-public.conservation.ca.gov/Well/Well/Detail?api=' + '04' + str(county).zfill(3) + str(unique).zfill(5))
            driver.switch_to.window(driver.window_handles[0])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            #Check to see if page contains error message, quit if so
            try:
                if (driver.find_element_by_xpath('//*[@id="main-content-anchor"]/div/div/div[2]/ul/li[1]').text == 'An error occurred while processing your request.'):
                    incorrect = incorrect + 1
                    #driver.quit()
            except:
                incorrect = 0
                x = 0
                #Click through all of the buttons in the document, making all info available
                while x < 3:
                    try:
                        WebDriverWait(driver,0.25).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span.indicator.pull-right.ca-gov-icon-menu-toggle-closed')))
                        driver.find_element_by_css_selector('span.indicator.pull-right.ca-gov-icon-menu-toggle-closed').click()
                    except (NoSuchElementException,ElementNotVisibleException,WebDriverException):
                        x = x+1
                        pass

                #Store the row values in the entire document
                values.extend(driver.find_element_by_class_name('row').text.split('\n'))

                #Remove data that interferes with next step
                values.pop(values.index("Install Date"))
                values.pop(values.index("Remove Date"))

                #Pull information from values list and store it in correct order in ordered_values
                for i in labels:
                    try:
                        ordered_values.append(values[values.index(i)+1])
                        values.pop(values.index(i) + 1)
                        values.pop(values.index(i))
                    except ValueError:
                        ordered_values.append("/")
                        pass

                #Check to see if Wellhead Equipment table is empty
                try:
                    if (driver.find_element_by_xpath('//*[@id="WellheadEquipmentGrid"]/div[2]/table').text != ''):
                        ordered_values.append("Yes")
                    else:
                        ordered_values.append("No")
                except:
                    pass

                #Check to see if Extra Casing Information is available
                try:
                    if driver.find_element_by_xpath('//*[@id="wellbore0"]/div/div/div[2]/div/div[2]/div[3]/span').is_displayed() & driver.find_element_by_xpath('//*[@id="wellbore0"]/div/div/div[2]/div/div[1]/div/span').text:
                        ordered_values.append("Yes")
                except:
                    ordered_values.append("No")
                    pass

                #Extract Bottom Hole (MD) & (TVD) values
                for i in values:
                    if "Bottom Hole (MD):" in i:
                        ordered_values.append(i[17:])
                        break
                for i in values:
                    if "Bottom Hole (TVD):" in i:
                        ordered_values.append(i[18:])
                        break
                #Add state information (always CA) 
                ordered_values.append("CA")
                #Write data to .csv and quit the driver
                dictionary = dict(zip(excel_labels, ordered_values))
                dictionary["Timestamp"] = '{:%H:%M:%S}'.format(datetime.datetime.now())
                print(dictionary)
                writeData(dictionary)
            print("End")
        else:
            break
        if incorrect > 3:
            print("County Complete")
            break

def writeData(data):
    row = []
    for i in excel_labels:
        try:
            row.append(data[i])
        except KeyError:
            row.append('/')
            pass
    with open('WellInfo_v2.csv', 'a', newline='') as file:
        write = csv.writer(file, dialect='excel')
        write.writerow(row)

def initialize_Columns():
    with open('WellInfo_v2.csv', 'w', newline='') as file:
        write = csv.writer(file, dialect='excel')
        write.writerow(excel_labels)

def generateEntireDataset():
    initialize_Columns()
    for i in CA_counties:
        time.sleep(1)
        collectandWrite(i)
    print("Finished!")

    
#collectandWrite(13)   
generateEntireDataset()

0401300001


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=93.0.4577.82)
  (Driver info: chromedriver=2.42.591059 (a3d9684d10d61aa0c45f6723b327283be1ebaad8),platform=Mac OS X 10.16.0 x86_64)
